In [47]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [48]:
df = pd.read_csv("data/final/final.csv")
df = df[df['pot_doc'] != '[]']

y = np.array(df['documentid'])
X = np.array(df)

# df["pot_doc"].astype("O")
# df["pot_date"].astype("O")
# df["pot_price"].astype("O")
# df["pot_name"].astype("O")
# df["pot_address"].astype("O")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
x_column = list(df.columns)
x_column.remove('documentid')

X_train_df = pd.DataFrame(data = X_train, columns = df.columns)
X_test_df = pd.DataFrame(data = X_test, columns = df.columns)
y_train_df = pd.DataFrame(data = y_train, columns = ['documentid'])
y_test_df = pd.DataFrame(data = y_test, columns = ['documentid'])

X_train_df= X_train_df.drop(columns="documentid").drop(columns="Unnamed: 0")
X_test_df= X_test_df.drop(columns="documentid").drop(columns="Unnamed: 0")
display(X_test_df)

,pot_doc,pot_name,pot_price,pot_address,pot_date
0,['00d0757881141'],[0],[75],[96],[1]
1,"['00d0717257644', '00d0382500008', '00d0390492...","[8, 15, 13, 8, 13]","[80, 50, 100, 89, 60]","[96, 90, 100, 100, 81]","[0, 0, 0, 0, 0]"
2,['00d0710387537'],[16],[100],[96],[1]
3,['00d0757215171'],[0],[100],[94],[1]
4,"['00d0737345187', '00d0450998902']","[19, 20]","[67, 100]","[100, 90]","[0, 0]"
...,...,...,...,...,...
94,"['00d0330533884', '00d0145695018', '00d0955195...","[17, 17, 17, 11, 17, 11, 17, 17, 11, 11, 17, 1...","[100, 100, 100, 100, 50, 100, 100, 100, 100, 1...","[96, 88, 96, 96, 96, 96, 96, 92, 88, 92, 92, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
95,['00d0773577731'],[4],[75],[97],[1]
96,"['00d0834036599', '00d0935614053']","[13, 16]","[100, 50]","[90, 93]","[1, 1]"
97,['00d0568013096'],[15],[80],[93],[1]


In [62]:
def compute_accuracy(x, y, z,a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2)

            #+ (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return np.sum(results == np.array(y_train_df['documentid']))/len(X_train_df)
    
def compute_accuracy2(x, y, z, a):
    results = []
    for row in X_test_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return results

In [55]:

# dfs = pd.DataFrame()
# ss = pd.DataFrame()

# dfs['result'] = ss.apply(lambda row: (row['documentid'] in row['pot_doc']), axis=1)
# print(dfs)
# num_trues = dfs['result'].value_counts()[True]
# print(num_trues)
def objective(trial):
    x = trial.suggest_float('x', 0, 10)
    y = trial.suggest_float('y', 0, 10)
    #z = trial.suggest_float('z', 0, 1)
    #a = trial.suggest_float('a', 0, 10)
    return 1-compute_accuracy(x,y,1,0)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}



[I 2023-01-29 06:15:50,484] A new study created in memory with name: no-name-119df105-6f16-4a23-8841-7ebd458acb6a
[I 2023-01-29 06:15:50,647] Trial 0 finished with value: 0.1670886075949367 and parameters: {'x': 1.8786858543722296, 'y': 8.566932960575002}. Best is trial 0 with value: 0.1670886075949367.
[I 2023-01-29 06:15:50,807] Trial 1 finished with value: 0.1670886075949367 and parameters: {'x': 0.32404221977446745, 'y': 6.05607127307028}. Best is trial 0 with value: 0.1670886075949367.
[I 2023-01-29 06:15:50,965] Trial 2 finished with value: 0.1670886075949367 and parameters: {'x': 4.65010885351785, 'y': 7.755576456082353}. Best is trial 0 with value: 0.1670886075949367.
[I 2023-01-29 06:15:51,122] Trial 3 finished with value: 0.1670886075949367 and parameters: {'x': 7.674609637232703, 'y': 9.162668126173397}. Best is trial 0 with value: 0.1670886075949367.
[I 2023-01-29 06:15:51,280] Trial 4 finished with value: 0.1670886075949367 and parameters: {'x': 5.799043757618399, 'y': 4.4

{'x': 1.8786858543722296, 'y': 8.566932960575002}

In [52]:
y_pred = study.predict(X_test_df)
print(y_pred)

AttributeError: 'Study' object has no attribute 'predict'

In [63]:
res = compute_accuracy2(study.best_params['x'], 1, study.best_params['y'], 0)

real = list(y_test_df['documentid'])

print(len(res), len(real))

99 99


In [64]:
def calculate_percentage(list_a, list_b):
    common_elements = set(list_a) & set(list_b)
    percentage = (len(common_elements) / len(list_a)) * 100
    return percentage
print(calculate_percentage(res,real))

81.81818181818183


In [ ]:
c = 0
for i in range(len(res)):
    if res[i] != real[i] and real[i] in eval(X_train_df['pot_doc'][i]):
        print(res[i], real[i])
        c += 1
print(c)

00d0201738305 00d0568013096
00d0954032591 00d0628164309
00d0511840834 00d0734092781
00d0358154075 00d0982931302
00d0106074517 00d0967073777
00d0648982621 00d0627668053
00d0396141876 00d0375035482
00d0546591203 00d0734493835
00d0975647950 00d0530467663
00d0167381962 00d0977582300
00d0989196649 00d0630325043
00d0250566763 00d0619690945
00d0954032591 00d0452553939
00d0954032591 00d0880901672
00d0954032591 00d0970540567
00d0954032591 00d0639354292
00d0724139826 00d0242626168
00d0542653551 00d0944630849
00d0542888628 00d0485748096
00d0390492165 00d0259552507
00d0390492165 00d0717257644
00d0250566763 00d0997573916
00d0954032591 00d0558270937
00d0243046961 00d0542157895
00d0674212079 00d0742987288
00d0645069761 00d0417300467
00d0763327632 00d0432757654
00d0954032591 00d0408489357
00d0212522180 00d0177909385
00d0954032591 00d0602427476
00d0992127574 00d0394257241
00d0382500008 00d0805870344
00d0375035482 00d0763327632
00d0202365816 00d0551541808
00d0627357411 00d0500564944
00d0511840834 00d018